While editing this notebook, don't change cell types as that confuses the autograder.

Before you turn this notebook in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [2]:
NAME = "Carmen Pelayo Fernández"

_Understanding Deep Learning_

---

<a href="https://colab.research.google.com/github/DL4DS/sp2024_notebooks/blob/main/release/nbs12/12_3_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 12.3: Tokenization

This notebook builds set of tokens from a text string as in figure 12.8 of the book,
employing a simplified version of the byte pair encoding algorithm.

In [3]:
import re, collections

In [4]:
text = "a sailor went to sea sea sea "+\
                  "to see what he could see see see "+\
                  "but all that he could see see see "+\
                  "was the bottom of the deep blue sea sea sea"

In [5]:
print(text)
print(len(text))

a sailor went to sea sea sea to see what he could see see see but all that he could see see see was the bottom of the deep blue sea sea sea
139


## Helper Functions

To begin with, the tokens are the individual letters and the </w> whitespace token. To visualize this, we represent each word in terms of these tokens with spaces between the tokens to delineate them.

The tokenized text is stored in a structure that represents each word as tokens together with the count of how often that word occurs.  We'll call this the *vocabulary*.

In [6]:
def initialize_vocabulary(text):
  """
  Initialize the vocabulary for the BPE algorithm.
   
  Initialize the vocabulary from the text string we will be using to 'train' 
  the algorithm. The vocabulary is a dictionary.  The keys are the words in the
  text and the values are the frequency of the word in the text.

  Returns:
  vocab: A dictionary where the keys are the words in the text and the values
          are the frequency of the word in the text.
  """
  vocab = collections.defaultdict(int)
  words = text.strip().split()
  for word in words:
      vocab[' '.join(list(word)) + ' </w>'] += 1
  return vocab

In [7]:
vocab = initialize_vocabulary(text)
print('Vocabulary: {}'.format(vocab))
print('Size of vocabulary: {}'.format(len(vocab)))

Vocabulary: defaultdict(<class 'int'>, {'a </w>': 1, 's a i l o r </w>': 1, 'w e n t </w>': 1, 't o </w>': 2, 's e a </w>': 6, 's e e </w>': 7, 'w h a t </w>': 1, 'h e </w>': 2, 'c o u l d </w>': 2, 'b u t </w>': 1, 'a l l </w>': 1, 't h a t </w>': 1, 'w a s </w>': 1, 't h e </w>': 2, 'b o t t o m </w>': 1, 'o f </w>': 1, 'd e e p </w>': 1, 'b l u e </w>': 1})
Size of vocabulary: 18


Find all the tokens in the current vocabulary and their frequencies

In [8]:
def get_tokens_and_frequencies(vocab):
  """
  Get the tokens and their frequencies from the vocabulary.
  
  Returns:
  tokens: A dictionary where the keys are the tokens and the values are the
          frequency of the token in the vocabulary.
  """
  tokens = collections.defaultdict(int)
  for word, freq in vocab.items():
      word_tokens = word.split()
      for token in word_tokens:
          tokens[token] += freq
  return tokens

In [9]:
tokens = get_tokens_and_frequencies(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))

Tokens: defaultdict(<class 'int'>, {'a': 12, '</w>': 33, 's': 15, 'i': 1, 'l': 6, 'o': 8, 'r': 1, 'w': 3, 'e': 28, 'n': 1, 't': 11, 'h': 6, 'c': 2, 'u': 4, 'd': 3, 'b': 3, 'm': 1, 'f': 1, 'p': 1})
Number of tokens: 19


Find each pair of adjacent tokens in the vocabulary
and count them.  We will subsequently merge the most frequently occurring pair.

In [10]:
def get_pairs_and_counts(vocab):
    """
    Get the pairs of symbols and their counts from the vocabulary.
    
    Returns:
    pairs: A dictionary where the keys are the pairs of symbols and the values
            are the frequency of the pair in the vocabulary.
    """
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

In [11]:
pairs = get_pairs_and_counts(vocab)
print(f'Pairs: {pairs}')
print(f'Number of distinct pairs: {len(pairs)}')

most_frequent_pair = max(pairs, key=pairs.get)
print(f'Most frequent pair: {most_frequent_pair}')

Pairs: defaultdict(<class 'int'>, {('a', '</w>'): 7, ('s', 'a'): 1, ('a', 'i'): 1, ('i', 'l'): 1, ('l', 'o'): 1, ('o', 'r'): 1, ('r', '</w>'): 1, ('w', 'e'): 1, ('e', 'n'): 1, ('n', 't'): 1, ('t', '</w>'): 4, ('t', 'o'): 3, ('o', '</w>'): 2, ('s', 'e'): 13, ('e', 'a'): 6, ('e', 'e'): 8, ('e', '</w>'): 12, ('w', 'h'): 1, ('h', 'a'): 2, ('a', 't'): 2, ('h', 'e'): 4, ('c', 'o'): 2, ('o', 'u'): 2, ('u', 'l'): 2, ('l', 'd'): 2, ('d', '</w>'): 2, ('b', 'u'): 1, ('u', 't'): 1, ('a', 'l'): 1, ('l', 'l'): 1, ('l', '</w>'): 1, ('t', 'h'): 3, ('w', 'a'): 1, ('a', 's'): 1, ('s', '</w>'): 1, ('b', 'o'): 1, ('o', 't'): 1, ('t', 't'): 1, ('o', 'm'): 1, ('m', '</w>'): 1, ('o', 'f'): 1, ('f', '</w>'): 1, ('d', 'e'): 1, ('e', 'p'): 1, ('p', '</w>'): 1, ('b', 'l'): 1, ('l', 'u'): 1, ('u', 'e'): 1})
Number of distinct pairs: 48
Most frequent pair: ('s', 'e')


Merge the instances of the best pair in the vocabulary

In [12]:
def merge_pair_in_vocabulary(pair, vocab_in):
    """
    Merge a pair of symbols in the vocabulary.
    
    Returns:
    vocab_out: A dictionary where the keys are the updated tokens in the 
               vocabulary and the values are the frequency of the token in the
               vocabulary.
    """
    vocab_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in vocab_in:
        word_out = p.sub(''.join(pair), word)
        vocab_out[word_out] = vocab_in[word]
    return vocab_out

In [13]:
vocab = merge_pair_in_vocabulary(most_frequent_pair, vocab)
print(f'Vocabulary: {vocab}')
print(f'Size of vocabulary: {len(vocab)}')

Vocabulary: {'a </w>': 1, 's a i l o r </w>': 1, 'w e n t </w>': 1, 't o </w>': 2, 'se a </w>': 6, 'se e </w>': 7, 'w h a t </w>': 1, 'h e </w>': 2, 'c o u l d </w>': 2, 'b u t </w>': 1, 'a l l </w>': 1, 't h a t </w>': 1, 'w a s </w>': 1, 't h e </w>': 2, 'b o t t o m </w>': 1, 'o f </w>': 1, 'd e e p </w>': 1, 'b l u e </w>': 1}
Size of vocabulary: 18


Update the tokens, which now include the best token 'se'

In [14]:
tokens = get_tokens_and_frequencies(vocab)
print(f'Tokens: {tokens}')
print(f'Number of tokens: {tokens}')

Tokens: defaultdict(<class 'int'>, {'a': 12, '</w>': 33, 's': 2, 'i': 1, 'l': 6, 'o': 8, 'r': 1, 'w': 3, 'e': 15, 'n': 1, 't': 11, 'se': 13, 'h': 6, 'c': 2, 'u': 4, 'd': 3, 'b': 3, 'm': 1, 'f': 1, 'p': 1})
Number of tokens: defaultdict(<class 'int'>, {'a': 12, '</w>': 33, 's': 2, 'i': 1, 'l': 6, 'o': 8, 'r': 1, 'w': 3, 'e': 15, 'n': 1, 't': 11, 'se': 13, 'h': 6, 'c': 2, 'u': 4, 'd': 3, 'b': 3, 'm': 1, 'f': 1, 'p': 1})


## Byte Pair Encoding

Now let's write the full tokenization routine

In [17]:
# TODO -- write this routine by filling in this missing parts,
# calling the above routines
def tokenize(text, num_merges):
  # Initialize the vocabulary from the input text
  # vocab = (your code here)
  vocab = initialize_vocabulary(text)

  for i in range(num_merges):
    # Find the tokens and how often they occur in the vocabulary
    tokens = get_tokens_and_frequencies(vocab)
      
    # Find the pairs of adjacent tokens and their counts
    pairs = get_pairs_and_counts(vocab)

    # Find the most frequent pair
    most_frequent_pair = max(pairs, key=pairs.get)
    print(f'Iter {i}: Most frequent pair: {most_frequent_pair}')

    # Merge the code in the vocabulary
    vocab = merge_pair_in_vocabulary(most_frequent_pair, vocab)

  # Find the tokens and how often they occur in the vocabulary one last time
  tokens = get_tokens_and_frequencies(vocab)

  return tokens, vocab

In [18]:
tokens, vocab = tokenize(text, num_merges=22)

Iter 0: Most frequent pair: ('s', 'e')
Iter 1: Most frequent pair: ('e', '</w>')
Iter 2: Most frequent pair: ('a', '</w>')
Iter 3: Most frequent pair: ('se', 'e</w>')
Iter 4: Most frequent pair: ('se', 'a</w>')
Iter 5: Most frequent pair: ('t', '</w>')
Iter 6: Most frequent pair: ('h', 'e</w>')
Iter 7: Most frequent pair: ('t', 'o')
Iter 8: Most frequent pair: ('to', '</w>')
Iter 9: Most frequent pair: ('h', 'a')
Iter 10: Most frequent pair: ('ha', 't</w>')
Iter 11: Most frequent pair: ('c', 'o')
Iter 12: Most frequent pair: ('co', 'u')
Iter 13: Most frequent pair: ('cou', 'l')
Iter 14: Most frequent pair: ('coul', 'd')
Iter 15: Most frequent pair: ('could', '</w>')
Iter 16: Most frequent pair: ('t', 'he</w>')
Iter 17: Most frequent pair: ('s', 'a')
Iter 18: Most frequent pair: ('sa', 'i')
Iter 19: Most frequent pair: ('sai', 'l')
Iter 20: Most frequent pair: ('sail', 'o')
Iter 21: Most frequent pair: ('sailo', 'r')


In [19]:
print(f'Tokens: {tokens}')
print(f'Number of tokens: {len(tokens)}')
print(f'Vocabulary: {vocab}')
print(f'Size of vocabulary: {len(vocab)}')

Tokens: defaultdict(<class 'int'>, {'a</w>': 1, 'sailor': 1, '</w>': 6, 'w': 3, 'e': 3, 'n': 1, 't</w>': 2, 'to</w>': 2, 'sea</w>': 6, 'see</w>': 7, 'hat</w>': 2, 'he</w>': 2, 'could</w>': 2, 'b': 3, 'u': 2, 'a': 2, 'l': 3, 't': 2, 's': 1, 'the</w>': 2, 'o': 2, 'to': 1, 'm': 1, 'f': 1, 'd': 1, 'p': 1, 'e</w>': 1})
Number of tokens: 27
Vocabulary: {'a</w>': 1, 'sailor </w>': 1, 'w e n t</w>': 1, 'to</w>': 2, 'sea</w>': 6, 'see</w>': 7, 'w hat</w>': 1, 'he</w>': 2, 'could</w>': 2, 'b u t</w>': 1, 'a l l </w>': 1, 't hat</w>': 1, 'w a s </w>': 1, 'the</w>': 2, 'b o t to m </w>': 1, 'o f </w>': 1, 'd e e p </w>': 1, 'b l u e</w>': 1}
Size of vocabulary: 18


After 22 merges of the tokenizer, you should have 27 tokens and the vocabulary consisting of 18 different tokens.

In [20]:
# Test code. Do not edit.

assert len(vocab) == 18
assert len(tokens) == 27